<a href="https://colab.research.google.com/github/andreunilux/BSP_6/blob/main/Hyperparameter_Tuning__Bert_with_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install missing dependancies
!pip install transformers accelerate
!pip install optuna


#Library used for fine tuning

from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments

# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 
import datetime


# HuggingFace Libarary
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer, BertConfig, BertTokenizer
from transformers import EarlyStoppingCallback



#All tested models 
bert_cased="bert-base-cased"
bert="bert-base-uncased"
HateBert="GroNLP/hateBERT"
DistilBert="distilbert-base-uncased"
RoBERTa="roberta-base" #'eval_f1': 0.5518856032046312 on gold
HateRoBERTa="facebook/roberta-hate-speech-dynabench-r4-target"


# Model HyperParameters
current_model=RoBERTa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
gold='/content/drive/MyDrive/BSP6/init_dataset_gold'
gold_pre_processed = '/content/drive/MyDrive/BSP6/init_dataset_gold_preprocessed' # gold dataset without links


current_dataset=gold_pre_processed
os.chdir(current_dataset) 
now = str(datetime.datetime.now())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_data = pd.read_json('train.json', lines=True)
val_data = pd.read_json('val.json', lines=True)
test_data = pd.read_json('test.json', lines=True)

In [4]:
# load pre-trained HateBert
def model_init():
    if(current_model=="roberta-base"):
        model = AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=3)
        tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    elif(current_model=="facebook/roberta-hate-speech-dynabench-r4-target"):
        tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
        model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target",num_labels=3, ignore_mismatched_sizes=True,hidden_dropout_prob=0.3,attention_probs_dropout_prob=0.3)
    else:
        model = BertForSequenceClassification.from_pretrained(current_model,num_labels=3 )
        tokenizer = BertTokenizer.from_pretrained(current_model)
    return model

def tokenizer_init():
    if(current_model=="roberta-base"):
        tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    elif(current_model=="facebook/roberta-hate-speech-dynabench-r4-target"):
        tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
    else:
        tokenizer = BertTokenizer.from_pretrained(current_model)
    return tokenizer

In [5]:
model=model_init()
tokenizer=tokenizer_init()
model

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should pr

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [6]:
#Train Data
#max length var
max_length_X=225
max_length_V=225
max_length_T=225

#truncation var
tru = True

X = []
for context, target in zip(train_data.context, train_data.target):
  X.append(tokenizer(context, target, padding='max_length', truncation=tru, max_length=max_length_X))
y_train = list(train_data.label)
print(X)
X_train_tokenized = X


#val Data
V = []
for context, target in zip(val_data.context, val_data.target):
   V.append(tokenizer( context,target, padding='max_length', truncation=tru, max_length=max_length_V))
y_val = list(val_data.label)
V_val_tokenized = V

#test Data
T = []
for context, target in zip(test_data.context, test_data.target):
   T.append(tokenizer(context, target, padding='max_length', truncation=tru, max_length=max_length_T))
y_test = list(test_data.label)
T_test_tokenized = T





IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
#print the token
print(tokenizer.decode(X_train_tokenized[0].input_ids))
print(X_train_tokenized[0])
print(len(X_train_tokenized[0].input_ids))
print(y_train[0])


print(tokenizer.decode(X_train_tokenized[1].input_ids))
print(X_train_tokenized[1])
print(len(X_train_tokenized[1].input_ids))
print(y_train[1])


<s>The UK is fucked.</s></s>>The ~~UK~~ world is fucked  FTFY</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [8]:
from torch.utils.data import DataLoader, RandomSampler

# Create torch dataset
#need a dictionary
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids':torch.tensor(self.encodings[idx].input_ids),
                'attention_mask':torch.tensor(self.encodings[idx].attention_mask),
                'labels':torch.tensor(self.labels[idx])}
        
        return item

    def __len__(self):
        return len(self.encodings)

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(V_val_tokenized, y_val)
test_dataset = Dataset(T_test_tokenized, y_test)


In [9]:
train_dataset[0]

{'input_ids': tensor([    0,   133,   987,    16, 42647,     4,     2,     2, 15698,   133,
          1437, 48256, 10494, 48256,   232,    16, 42647,  1437,   274, 20249,
           975,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
f = open(current_dataset+"/Report/output"+now+".txt","w")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    confusion = confusion_matrix(labels, preds)
    print('Confusion Matrix\n')
    print(confusion)
    print('\nClassification Report\n')
    print(classification_report(labels, preds, target_names=['Hate', 'Neutral', 'Counter']))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"eval_f1": f1, "accuracy": accuracy, "precision": precision, "recall": recall}



In [11]:

# Define Trainer
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,               # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    learning_rate = 2e-5,
    evaluation_strategy="steps",
    logging_steps = 50,
    weight_decay=0.1,
    load_best_model_at_end=True,
    overwrite_output_dir=True
)            



trainer = Trainer(
    model_init=model_init,  # now the trainer has a model_init method
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

best_trial = trainer.hyperparameter_search(
    direction="maximize", 
    backend="optuna", 
    n_trials=20,  # number of trials
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should pr

Step,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
50,1.073000,1.036099,0.332554,0.499299,0.249299,0.499299
100,1.028900,0.995918,0.462530,0.518934,0.479892,0.518934
150,1.007600,0.969908,0.437902,0.545582,0.424840,0.545582
200,0.987000,0.952776,0.542504,0.556802,0.548849,0.556802
250,0.938200,0.944685,0.526783,0.542777,0.576567,0.542777
300,0.843100,0.975616,0.530315,0.562412,0.559280,0.562412
350,0.846700,0.965550,0.572278,0.566620,0.585282,0.566620
400,0.866000,0.918063,0.577366,0.604488,0.581734,0.604488


Confusion Matrix

[[  0 202   0]
 [  0 356   0]
 [  0 155   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.00      0.00      0.00       202
     Neutral       0.50      1.00      0.67       356
     Counter       0.00      0.00      0.00       155

    accuracy                           0.50       713
   macro avg       0.17      0.33      0.22       713
weighted avg       0.25      0.50      0.33       713



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Confusion Matrix

[[ 88 109   5]
 [ 75 277   4]
 [ 59  91   5]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.40      0.44      0.42       202
     Neutral       0.58      0.78      0.67       356
     Counter       0.36      0.03      0.06       155

    accuracy                           0.52       713
   macro avg       0.44      0.42      0.38       713
weighted avg       0.48      0.52      0.46       713

Confusion Matrix

[[ 45 157   0]
 [ 12 344   0]
 [ 27 128   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.54      0.22      0.31       202
     Neutral       0.55      0.97      0.70       356
     Counter       0.00      0.00      0.00       155

    accuracy                           0.55       713
   macro avg       0.36      0.40      0.34       713
weighted avg       0.42      0.55      0.44       713



...

Confusion Matrix

[[  0 202   0]
 [  0 356   0]
 [  0 155   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.00      0.00      0.00       202
     Neutral       0.50      1.00      0.67       356
     Counter       0.00      0.00      0.00       155

    accuracy                           0.50       713
   macro avg       0.17      0.33      0.22       713
weighted avg       0.25      0.50      0.33       713



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Confusion Matrix

[[  0 202   0]
 [  0 356   0]
 [  0 155   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.00      0.00      0.00       202
     Neutral       0.50      1.00      0.67       356
     Counter       0.00      0.00      0.00       155

    accuracy                           0.50       713
   macro avg       0.17      0.33      0.22       713
weighted avg       0.25      0.50      0.33       713



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 26>:26                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2652 in hyperparameter_search    │
│                                                                                                  │
│   2649 │   │   │   HPSearchBackend.SIGOPT: run_hp_search_sigopt,                                 │
│   2650 │   │   │   HPSearchBackend.WANDB: run_hp_search_wandb,                                   │
│   2651 │   │   }                                                                                 │
│ ❱ 2652 │   │   best_run = backend_dict[backend](self, n_trials, direction, **kwargs)             │
│   2653 │   │                                                                                     │
│   2654 │   │   self.hp_search_backend = None                                                     │
│   2655 │   │   return best_run                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/integrations.py:218 in run_hp_search_optuna │
│                                                                                                  │
│    215 │   │   timeout = kwargs.pop("timeout", None)                                             │
│    216 │   │   n_jobs = kwargs.pop("n_jobs", 1)                                                  │
│    217 │   │   study = optuna.create_study(direction=direction, **kwargs)                        │
│ ❱  218 │   │   study.optimize(_objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)     │
│    219 │   │   best_trial = study.best_trial                                                     │
│    220 │   │   return BestRun(str(best_trial.number), best_trial.value, best_trial.params)       │
│    221 │   else:                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/study.py:443 in optimize                    │
│                                                                                                  │
│    440 │   │   │   │   If nested invocation of this method occurs.                               │
│    441 │   │   """                                                                               │
│    442 │   │                                                                                     │
│ ❱  443 │   │   _optimize(                                                                        │
│    444 │   │   │   study=self,                                                                   │
│    445 │   │   │   func=func,                                                                    │
│    446 │   │   │   n_trials=n_trials,                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:66 in _optimize                │
│                                                                                                  │
│    63 │                                                                                          │
│    64 │   try:                                                                                   │
│    65 │   │   if n_jobs == 1:                                                                    │
│ ❱  66 │   │   │   _optimize_sequential(                                                          │
│    67 │   │   │   │   study,                               

In [ ]:


f.write("model"+ current_model)
f.write("dataset: "+ current_dataset)
f.write("\n")
f.write("structure of the model: \n")
f.write(str(trainer.model))
f.write("\n")
f.write("Tokenizer max length train:"+str(max_length_X))
f.write("Tokenizer max length val:"+str(max_length_V))
f.write("Tokenizer max length test:"+str(max_length_T))
f.write("\n")
f.write(str(trainer.args))
f.write("\n")

In [ ]:
trainer.train()
f.write(str(trainer.state.log_history))
eval=trainer.evaluate(test_dataset)

In [ ]:
eval

In [ ]:
f.close()